In [69]:
import pandas as pd
import csv
import os

# Parse Master
Parses all census tables via the intermediary form and combines them into a master file with `cfips` and `year`. Also brings in density data from `density.csv`.

In [70]:
# Parse `density.csv` first
df = pd.read_csv('./density.csv')
df['year'] = df['first_day_of_month'].apply(lambda x: x[:4])
df = df.groupby(['year', 'cfips', 'county', 'state']).mean(numeric_only = True).reset_index()
df = df.filter(items=['year', 'cfips', 'county', 'state', 'microbusiness_density'])
df

,year,cfips,county,state,microbusiness_density
0,2019,1001,Autauga County,Alabama,2.986972
1,2019,1003,Baldwin County,Alabama,7.370375
2,2019,1005,Barbour County,Alabama,1.046068
3,2019,1007,Bibb County,Alabama,1.278288
4,2019,1009,Blount County,Alabama,1.559113
...,...,...,...,...,...
12535,2022,56037,Sweetwater County,Wyoming,2.840650
12536,2022,56039,Teton County,Wyoming,25.778021
12537,2022,56041,Uinta County,Wyoming,3.954946
12538,2022,56043,Washakie County,Wyoming,3.088503


In [71]:
# Grab all intermediate files and add in
for path in os.listdir('./census-data'):
    table = pd.DataFrame()
    for file in os.listdir('./census-data/' + path + '/parsed'):
        year = file[:4]
        temp = pd.read_csv('./census-data/' + path + '/parsed/' + file)

        temp['year'] = year
        table = pd.concat([table, temp])
    df = pd.merge(df, table, how="left", on=['cfips', 'year'])

df.head()
df.to_csv('master.csv', index=False)

,year,cfips,county,state,microbusiness_density,pct_unemployed,pct_broadband,pct_college,pct_it_workers,median_hh_inc
0,2019,1001,Autauga County,Alabama,2.986972,3.7,80.6,16.1,0.7,58731.0
1,2019,1003,Baldwin County,Alabama,7.370375,4.3,81.8,21.0,1.4,58320.0
2,2019,1005,Barbour County,Alabama,1.046068,9.2,60.5,7.6,0.8,32525.0
3,2019,1007,Bibb County,Alabama,1.278288,7.3,69.2,6.5,1.6,47542.0
4,2019,1009,Blount County,Alabama,1.559113,3.4,73.0,8.6,0.9,49358.0
...,...,...,...,...,...,...,...,...,...,...
12535,2022,56037,Sweetwater County,Wyoming,2.840650,NaN,NaN,NaN,NaN,NaN
12536,2022,56039,Teton County,Wyoming,25.778021,NaN,NaN,NaN,NaN,NaN
12537,2022,56041,Uinta County,Wyoming,3.954946,NaN,NaN,NaN,NaN,NaN
12538,2022,56043,Washakie County,Wyoming,3.088503,NaN,NaN,NaN,NaN,NaN
